![](escudo.png)
    
# Analisis masivos de datos
## Contador SQL de la velocidad de conexion a internet
### Jose Gabriel Rodriguez Rivas
## Jorge Alberto Hernandez Castro


### Paso 1.- Conectarse a Base de Datos

In [1]:
# this is the library to talk to the database
import sqlite3
import pandas as pd
# this is a library for data vizualization
from matplotlib import pyplot as plt
# this is to say that we want our plots to be part of this page, and not opened in a new window
%matplotlib inline

In [2]:
conn = sqlite3.connect('InternetSpeed.db')

In [3]:
conn.cursor()

In [4]:
cur = conn.cursor()

In [5]:
consulta = 'SELECT DateTime FROM LA_wifi_speed_UK LIMIT 10;'
cur.execute(consulta)

In [6]:
for row in cur:
    print (row)

('2016-11-23 00:00:00.000000',)
('2016-11-23 00:30:00.000000',)
('2016-11-23 01:00:00.000000',)
('2016-11-23 01:30:00.000000',)
('2016-11-23 02:00:00.000000',)
('2016-11-23 02:30:00.000000',)
('2016-11-23 03:00:00.000000',)
('2016-11-23 03:30:00.000000',)
('2016-11-23 04:00:00.000000',)
('2016-11-23 04:30:00.000000',)


In [7]:
consulta = 'SELECT DateTime FROM LA_wifi_speed_UK ORDER BY RANDOM() LIMIT 10;'
cur.execute(consulta)

In [8]:
for row in cur:
    print (row)

('2016-12-10 08:00:00.000000',)
('2016-12-06 03:30:00.000000',)
('2016-12-11 00:30:00.000000',)
('2016-11-28 05:30:00.000000',)
('2016-12-10 05:00:00.000000',)
('2016-12-05 11:00:00.000000',)
('2016-12-09 08:30:00.000000',)
('2016-11-30 06:30:00.000000',)
('2016-12-03 12:30:00.000000',)
('2016-12-05 03:00:00.000000',)


In [9]:
consulta = 'SELECT * FROM LA_wifi_speed_UK LIMIT 1'
cur.execute(consulta)
cur.description[:10]

(('E07000223_p', None, None, None, None, None, None),
 ('E07000026_p', None, None, None, None, None, None),
 ('E07000032_p', None, None, None, None, None, None),
 ('E07000224_p', None, None, None, None, None, None),
 ('E07000170_p', None, None, None, None, None, None),
 ('E07000105_p', None, None, None, None, None, None),
 ('E07000004_p', None, None, None, None, None, None),
 ('E07000200_p', None, None, None, None, None, None),
 ('E09000002_p', None, None, None, None, None, None),
 ('E09000003_p', None, None, None, None, None, None))

In [10]:
# loop over the header and store them in a list
columns = [member[0] for member in cur.description]

In [11]:
# ignore index column
columns = columns[1:]

In [14]:
# remove suffix '_p'
columns = [c.replace('_p', '') for c in columns]
# remove suffix '_d'
columns = [c.replace('_d', '') for c in columns]
# remove suffix '_u'
columns = [c.replace('_u', '') for c in columns]
# this operation is to remove duplicates
columns = list(set(columns))

In [15]:
# visualize the first 10 areas
columns[:10]

['E06000049',
 'E07000192',
 'E08000005',
 'E06000018',
 'E07000088',
 'E07000194',
 'E07000038',
 'E07000220',
 'E07000110',
 'E09000026']

In [16]:
area = columns[0]
print (area)

E06000049


In [17]:
##'llave':'valor'
suffix = {'_p':'ping', '_d':'download', '_u':'upload'}
# we have the suffixes as the keys of the dictionary
print (suffix.keys() )

# we have the complete names as the values of the dictionary
print (suffix.values() )

dict_keys(['_p', '_d', '_u'])
dict_values(['ping', 'download', 'upload'])


In [18]:
area = columns[0]

area

'E06000049'

In [ ]:
## , para unir tablas
area = columns[0]
plt.figure(figsize=(10, 8))

# Plot each variable in suffix.keys() for each area
for s in suffix.keys():
    consulta = 'SELECT "{}{}" FROM LA_wifi_speed_UK ORDER BY DateTime'.format(area, s)
    cur.execute(consulta)
    plt.plot(list(cur.fetchall()), label=suffix[s])
plt.legend()
plt.title(area)

![](DUP.png)

## Parte 2 Manipular los datos con pandas

In [ ]:
del(df)

In [19]:
new_columns = ['Area', 'Ping_promedio', 'Descarga_promedio', 'Carga_promedio']
df = pd.DataFrame(columns = new_columns)
df

,Area,Ping_promedio,Descarga_promedio,Carga_promedio


In [20]:
# this will take a while...
for i in range(len(columns)-1):
    tmp_list = []
    tmp_list.append(columns[i])
    for s in suffix.keys():
        consulta = 'SELECT AVG("{}{}") FROM LA_wifi_speed_UK'.format(columns[i], s)
        cur.execute(consulta)
        mean = cur.fetchone()
        tmp_list.append(mean[0])
    #append the columns to the empty DataFrame
    df = df.append(pd.Series(tmp_list, index=new_columns), ignore_index=True)
# visualize the head of the dataframe here
df.head()

,Area,Ping_promedio,Descarga_promedio,Carga_promedio
0,E06000049,57.118931,23.482727,9.389337
1,E07000192,52.211869,20.804882,8.344268
2,E08000005,54.087219,22.189738,8.778285
3,E06000018,55.543339,23.018471,9.146285
4,E07000088,52.159555,20.821356,8.316713


In [21]:
len(df)

326

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(df.index, df[['Descarga_promedio','Carga_promedio','Ping_promedio']], 'o')

![](E08000021.png)

In [22]:
try:
    cur.execute('DROP TABLE average_speed')
except:
    pass
df.to_sql('average_speed', conn)

In [23]:
# just checking everything worked fine: select everything from the table we just created
# and then print the first two rows of the result

query_2 = 'SELECT * FROM average_speed'
cur.execute(query_2)

# print the first fetched record
print(cur.fetchone())
# print the next fetched record
print(cur.fetchone())

(0, 'E06000049', 57.11893112511649, 23.48272682151498, 9.38933695300067)
(1, 'E07000192', 52.21186919385844, 20.804881611149966, 8.34426803535672)


## Unir Tablas

In [24]:
query = 'SELECT * FROM LA_population LIMIT 10'
cur.execute(query)
for row in cur:
    print(row)

('Aberdeen City', 'S12000033', 222793)
('Aberdeenshire', 'S12000034', 252973)
('Adur', 'E07000223', 61182)
('Allerdale', 'E07000026', 96422)
('Amber Valley', 'E07000032', 122309)
('Angus', 'S12000041', 115978)
('Antrim and Newtownabbey', 'N09000001', 138567)
('Ards and North Down', 'N09000011', 156672)
('Argyll and Bute', 'S12000035', 88166)
('Armagh City, Banbridge and Craigavon', 'N09000002', 199693)


In [25]:
query = 'SELECT * FROM average_speed JOIN LA_population ON LA_population."LA_code"=average_speed.Area'

cur.execute(query)
k = 0
for row in cur:
    if k>10:
        break
    print(row)
    k+=1

(0, 'E06000049', 57.11893112511649, 23.48272682151498, 9.38933695300067, 'Cheshire East', 'E06000049', 370127)
(1, 'E07000192', 52.21186919385844, 20.804881611149966, 8.34426803535672, 'Cannock Chase', 'E07000192', 97462)
(2, 'E08000005', 54.08721859142486, 22.18973793152207, 8.7782853671417, 'Rochdale', 'E08000005', 211699)
(3, 'E06000018', 55.54333923008997, 23.01847059824432, 9.146285112583122, 'Nottingham', 'E06000018', 305680)
(4, 'E07000088', 52.159555077647454, 20.821356161137263, 8.31671318145464, 'Gosport', 'E07000088', 82622)
(5, 'E07000194', 52.11103901321929, 20.892160562907634, 8.355531143729888, 'Lichfield', 'E07000194', 100654)
(6, 'E07000038', 51.78235350498569, 20.860655087433024, 8.356288120951067, 'North East Derbyshire', 'E07000038', 99023)
(7, 'E07000220', 52.173417604189716, 20.96435683418015, 8.37447670333847, 'Rugby', 'E07000220', 100075)
(8, 'E07000110', 52.850341015191695, 21.72109968087522, 8.568198903188927, 'Maidstone', 'E07000110', 155143)
(9, 'E09000026',